Copyright (c) Microsoft Corporation. 
Licensed under the MIT license. 
## Recommendation Refresh
---
This script uses the trained ALS model to compute recommendations for targeted users who have new data or signals since the last run of the script.  


In [4]:
from pyspark.ml.recommendation import ALSModel
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import  StringIndexer, IndexToString

In [ ]:
try:
    spark.sql("CREATE DATABASE retailaidb")
except:
    print("Database already exists")

In [5]:
model_reload = ALSModel.load("retailai_recommendation_model")

In [7]:
get_top_users = spark.read.table("retailaidb.cleaned_dataset").groupBy(["user_id", "user_session"]).count().groupBy(["user_id"]).count().orderBy("count", ascending=False)

In [8]:
top_user_ids = get_top_users.select("user_id").limit(10)

In [9]:
preds_alt = model_reload.recommendForUserSubset(top_user_ids, 5).select("user_id", col("recommendations"))

In [29]:
user_preds_df = preds_alt.select("user_id",explode("recommendations").alias("recommendations")) \
                                    .select("user_id", "recommendations.product_id", "recommendations.rating").orderBy("rating", ascending=False)

In [31]:
final_recommendations = get_prod_data_df.groupBy("user_id").agg(collect_set("product_id").alias("product_ids")) \
                                                    .select("user_id", col("product_ids").cast("string"))

In [33]:
final_recommendations.withColumn("updated_ts", lit(current_timestamp()).cast("timestamp")) \
                                    .select(col("user_id").cast("string").alias("user_id"), "product_ids", "updated_ts") \
                                    .write.saveAsTable("retailaidb.top_shopper_recommendation_summary", mode="overwrite")